In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from cv2 import CascadeClassifier
import time
import PIL
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
from IPython.display import display, Image, clear_output
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils
from mtcnn import MTCNN

%pylab inline

Создание набора данных

In [ ]:
# Импорт и просмотр изображения
def viewImage(image, proba): 
    cv2.namedWindow(proba, cv2.WINDOW_NORMAL)
    cv2.imshow(proba, image)
    # cv2.waitKey(0) # добавить, если надо просматривать фото
    cv2.destroyAllWindows()

In [ ]:
# В папке pic находится набор фотографий разного размера с разным количеством людей на них.
pictures = os.listdir('pic')
face_detector = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')
n = 0
k = 0
while k < len(pictures):
    # считываем по порядку все изображения
    image = cv2.imread('pic/' + pictures[k])
    if type(image) == np.ndarray:
        image_copy = image.copy()
        # при распознавании установим следующие параметры:
        faces = face_detector.detectMultiScale(image, scaleFactor= 1.3, minNeighbors = 15, minSize=(60, 60))
        # Добавим столбец в ndarray faces для создания массива с номером фото
        column_to_be_added = np.array([i for i in range(1, len(faces) + 1)])
        faces_result = np.column_stack((faces, column_to_be_added))
        # создаем набор лиц
        for (x, y, w, h, i) in faces_result:
            # Вырезаем лица с нужным отступом      
            cropped = image[y: y+h+10, x + int(w/2-(h+10)/2): x + int(w/2 + (h+10)/2)]           
            # Приводим все фотографии к одному размеру
            img_resize = cv2.resize(cropped, (128, 128), interpolation = cv2.INTER_AREA)
            # Сохраняем в новой папке
            cv2.imwrite('result/''pic'+str(n + k + i)+'.jpg', img_resize)         
        print('на фото №', k+1, 'выбрано', i, 'лиц' )
        n += len(faces) - 1    
    else:        
        print('error  foto/' + pictures[k])
    k += 1

Создаем dataframe, содержащий имя фотографии, пол и возраст изображенного человека

In [ ]:
# Определить лицо и нарисовать ограничивающую рамку лица
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]  # Высота - это количество строк в матрице
    frameWidth = frameOpencvDnn.shape[1]  # Ширина - это количество столбцов в матрице
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    net.setInput(blob)
    detections = net.forward()  # Сеть проводит прямое распространение и обнаруживает лица
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])  # координаты ограничивающей рамки
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)),
                         8)  # rectangle(img, pt1, pt2, color[, thickness[, lineType[, shift]]]) -> img
    return frameOpencvDnn, bboxes

In [ ]:
# Сетевая модель и модель предварительного обучения
faceProto = "weights/opencv_face_detector.pbtxt"
faceModel = "weights/opencv_face_detector_uint8.pb"
ageProto = "weights/deploy_age.prototxt"
ageModel = "weights/age_net.caffemodel"
genderProto = "weights/gender_deploy.prototxt"
genderModel = "weights/gender_net.caffemodel"
# Модель означает
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']
# Загрузить сеть
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
# Сеть и модель распознавания лиц
faceNet = cv2.dnn.readNet(faceModel, faceProto)

In [ ]:
# Создаем пустой датасет
df_title = {'foto': [], 'gender': [], 'age': []}
df = pd.DataFrame(df_title)
df.head()

In [ ]:
# Создадим набор данных, соответствующий полу и возрастному промежутку рассматриваемой модели
pictures = os.listdir('result')
k = 0
padding = 20
t = time.time()
# Открыть фотографию
while k < len(pictures):
    frame = cv2.imread('result/' + pictures[k], cv2.IMREAD_UNCHANGED)
    frameFace, bboxes = getFaceBox(faceNet, frame)
    for bbox in bboxes:
        # Извлечь лицо, обрамленное рамкой для обнаружения, и вернуть изображение лица
        face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
                max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
        list_face = [] 
        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)   # blob входит в сеть для определения пола
        genderPreds = genderNet.forward()   # Определение пола для прямой передачи
        gender = genderList[genderPreds[0].argmax()]   # Категория Вернуться к типу пола
        if gender == 'Male':
            gender_name = '1'
        else:
            gender_name = '0'
        print("Фото №", k+1)
        print("Пол : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))
        ageNet.setInput(blob)
        agePreds = ageNet.forward()  # Определение возраста для прямой передачи
        age = ageList[agePreds[0].argmax()] # Категория Вернуться к типу возраста
        for j in range(len(ageList)):
            if age == ageList[j]:
                if j < 3:
                    age_name = str(0)
                    print('класс 0 - 0-16 - детство')
                if j == 3:
                    age_name = str(1)
                    print('класс 1 - 17-34 - молодость')
                if j == 4:
                    age_name = str(2)
                    print('класс 2 - 35-60 - зрелость')
                if j > 4:
                    age_name = str(3)
                    print('класс 3 - 60+ - пожилой возраст')
        print("Возраст : {}, conf = {:.3f}".format(age, agePreds[0].max()))
        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)  
        cv2.imwrite('res_finish_1/pic'+str(k+1)+'.jpg', frameFace)
        list_face.append({'foto': pictures[k], 'gender': gender_name, 'age': age_name})
        df = df.append(list_face) # добавление данных в датасет
    print("time : {:.3f} ms".format(time.time() - t))
    k += 1
print(gender_name)
print(age_name)

In [ ]:
# Проверка классов датасета
df['gender'].value_counts()

In [ ]:
df['age'].value_counts()

In [ ]:
# Создание csv-файла
df.to_csv('foto_frame_1.csv', index=False)

In [ ]:
# Открытие csv как dataframe
df = pd.read_csv('foto_frame_1.csv')
df

Создание модели сверточной нейронной сети для распознавания возраста

In [ ]:
# Считываем изображения по имени, сохраняем в массив данные
def foto_to_array(x_array, size):
    x = np.zeros((len(x_array), size, size))  
    i = 0
    for name in x_array:
        image = cv2.imread('result/' + name)
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        x[i] = np.array(gray_img)
        i += 1       
    return x

In [ ]:
x_new = foto_to_array(df['foto'].values, 128)
y_new = df['age'].values

In [ ]:
#Делим тестовую и обучающую выборки
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size = 0.25, random_state = 123)

# В нашем случае входными значениями являются пиксели в изображении, которые имеют значение от 0 до 255.
# Если значения входных данных находятся в слишком широком диапазоне, это может отрицательно повлиять на работу сети. 
# Желательно нормализовать данные
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# При определении класса применяем "двоичную классификацию": изображение либо принадлежит одному определённому классу, либо нет
# Для унитарного кодирования используется команда Numpy to_categorical(). 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [ ]:
# количество классов, которые будем определять
num_classes = 4

In [ ]:
# Создаем пустую модель. Sequential — классическая модель в Keras:
model_age = Sequential()

# Первый слой модели - это сверточный слой. Принимает входные данные и пропускать их через сверточные фильтры.
# - Количество каналов (фильтров):  32, 
# - размер фильтра:  (3 x 3), 
# - задаем форму входа (100*100 - размер фото, 1 - градации серого), 
# - отступы: padding = 'same' (то есть, мы не меняем размер изображения)
# - функция активации: relu 

model_age.add(Conv2D(32, (3, 3), input_shape = (128, 128, 1), activation = 'relu', padding='same'))

# Исключающий слой для предотвращения переобучения, который случайным образом устраняет соединения между слоями 
# 0,2 означает, что он отбрасывает 20% существующих соединений:

model_age.add(Dropout(0.2))

# Пакетная нормализация входных данных, поступающих в следующий слой. 
# Т.о. сеть всегда создает функции активации с тем же распределением, которое нам нужно:

model_age.add(BatchNormalization())

# Следующий сверточный слой, но размер фильтра увеличивается (сеть уже может изучать более сложные представления):

model_age.add(Conv2D(64, (3, 3), activation = 'relu', padding='same'))

# Основа рабочего процесса в первой части реализации CNN: свертка, активация, исключение, объединение.
# Следующий шаг:
# - Объединяющий слой - помогает сделать классификатор изображений более корректным, чтобы он мог изучать релевантные шаблоны. 
# - Исключение (Dropout) 
# - Пакетная нормализация

model_age.add(MaxPooling2D(pool_size=(2, 2)))
model_age.add(Dropout(0.2))
model_age.add(BatchNormalization())

# Повторяем эти слои:

model_age.add(Conv2D(128, (3, 3), activation = 'relu', padding='same'))
model_age.add(MaxPooling2D(pool_size=(2, 2)))
model_age.add(Dropout(0.2))
model_age.add(BatchNormalization())

# После того, как закончили со сверточными слоями,нужно сжать данные. (и добавляем слой исключения снова)

model_age.add(Flatten())
model_age.add(Dropout(0.2))

# Cоздаем первый плотно связанный слой (указываем количество нейронов в плотном слое)
# Число нейронов в последующих слоях будет теперь уменьшаться, 
# в конечном итоге приближаясь к тому же числу нейронов, что и классы в наборе данных (в данном случае 3). 
# maxnorm - ограничение ядра может упорядочить данные в процессе обучения, помогает предотвратить переобучение.

model_age.add(Dense(256, kernel_constraint=maxnorm(3)))
model_age.add(Activation('relu'))
model_age.add(Dropout(0.2))
model_age.add(BatchNormalization())

model_age.add(Dense(128, kernel_constraint=maxnorm(3)))
model_age.add(Activation('relu'))
model_age.add(Dropout(0.2))
model_age.add(BatchNormalization())

# В последнем слое мы уравниваем количество классов с числом нейронов. 
# Каждый нейрон представляет класс, поэтому на выходе этого слоя будет вектор из 3 нейронов, 
# каждый из которых хранит некоторую вероятность того, что рассматриваемое изображение принадлежит его классу.
# Функция активации softmax выбирает нейрон с наибольшей вероятностью в качестве своего выходного значения, 
# предполагая, что изображение принадлежит именно этому классу:

model_age.add(Dense(num_classes))
model_age.add(Activation('softmax'))

# количество эпох для обучения = ?
# оптимизатор - настроит веса в сети так, чтобы приблизиться к точке с наименьшими потерями. 
# Алгоритм Адама является одним из наиболее часто используемых оптимизаторов


optimizer = 'adam'

# компилируем модель с выбранными параметрами. метрику для оценки = accuracy

model_age.compile(loss ='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
print(model_age.summary())

In [ ]:
# SEED (симметричный блочный криптоалгоритм на основе Сети Фейстеля)
seed = 21
numpy.random.seed(seed)

In [ ]:
# Задаем число эпох обучения
epochs = 100
# epochs = 300

In [ ]:
numpy.random.seed(seed)
model_history_age = model_age.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=epochs, batch_size=64)

In [ ]:
# Точность модели
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# График точности модели
plt.plot(model_history_age.history['accuracy'])
plt.plot(model_history_age.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show

In [ ]:
# График функции потерь модели
plt.plot(model_history_age.history['loss'])
plt.plot(model_history_age.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show

In [ ]:
# Сохранение модели
from keras.models import load_model
model_age.save('model_age.h5')

Создание модели сверточной нейронной сети для распознавания пола

In [ ]:
x_new = foto_to_array(df['foto'].values, 128)
z_new = df['gender'].values

#Делим тестовую и обучающую выборки
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_new, z_new, test_size = 0.25, random_state = 123)

# В нашем случае входными значениями являются пиксели в изображении, которые имеют значение от 0 до 255.
# Если значения входных данных находятся в слишком широком диапазоне, это может отрицательно повлиять на работу сети. 
# Желательно нормализовать данные
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# При определении класса применяем "двоичную классификацию": изображение либо принадлежит одному определённому классу, либо нет
# Для унитарного кодирования используется команда Numpy to_categorical(). 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

# количество классов, которые будем определять
num_classes = 2

# Создаем пустую модель. Sequential — классическая модель в Keras:
model_gender = Sequential()

# Первый слой модели - это сверточный слой. Принимает входные данные и пропускать их через сверточные фильтры.
# - Количество каналов (фильтров):  32, 
# - размер фильтра:  (3 x 3), 
# - задаем форму входа (100*100 - размер фото, 1 - градации серого), 
# - отступы: padding = 'same' (то есть, мы не меняем размер изображения)
# - функция активации: relu 

model_gender.add(Conv2D(32, (3, 3), input_shape = (128, 128, 1), activation = 'relu', padding='same'))

# Исключающий слой для предотвращения переобучения, который случайным образом устраняет соединения между слоями 
# 0,2 означает, что он отбрасывает 20% существующих соединений:

model_gender.add(Dropout(0.2))

# Пакетная нормализация входных данных, поступающих в следующий слой. 
# Т.о. сеть всегда создает функции активации с тем же распределением, которое нам нужно:

model_gender.add(BatchNormalization())

# Следующий сверточный слой, но размер фильтра увеличивается (сеть уже может изучать более сложные представления):

model_gender.add(Conv2D(64, (3, 3), activation = 'relu', padding='same'))

# Основа рабочего процесса в первой части реализации CNN: свертка, активация, исключение, объединение.
# Следующий шаг:
# - Объединяющий слой - помогает сделать классификатор изображений более корректным, чтобы он мог изучать релевантные шаблоны. 
# - Исключение (Dropout) 
# - Пакетная нормализация

model_gender.add(MaxPooling2D(pool_size=(2, 2)))
model_gender.add(Dropout(0.2))
model_gender.add(BatchNormalization())

# Повторяем эти слои:

model_gender.add(Conv2D(128, (3, 3), activation = 'relu', padding='same'))
model_gender.add(MaxPooling2D(pool_size=(2, 2)))
model_gender.add(Dropout(0.2))
model_gender.add(BatchNormalization())

# После того, как закончили со сверточными слоями,нужно сжать данные. (и добавляем слой исключения снова)

model_gender.add(Flatten())
model_gender.add(Dropout(0.2))

# Cоздаем первый плотно связанный слой (указываем количество нейронов в плотном слое)
# Число нейронов в последующих слоях будет теперь уменьшаться, 
# в конечном итоге приближаясь к тому же числу нейронов, что и классы в наборе данных (в данном случае 3). 
# maxnorm - ограничение ядра может упорядочить данные в процессе обучения, помогает предотвратить переобучение.

model_gender.add(Dense(256, kernel_constraint=maxnorm(3)))
model_gender.add(Activation('relu'))
model_gender.add(Dropout(0.2))
model_gender.add(BatchNormalization())

model_gender.add(Dense(128, kernel_constraint=maxnorm(3)))
model_gender.add(Activation('relu'))
model_gender.add(Dropout(0.2))
model_gender.add(BatchNormalization())

# В последнем слое мы уравниваем количество классов с числом нейронов. 
# Каждый нейрон представляет класс, поэтому на выходе этого слоя будет вектор из 3 нейронов, 
# каждый из которых хранит некоторую вероятность того, что рассматриваемое изображение принадлежит его классу.
# Функция активации softmax выбирает нейрон с наибольшей вероятностью в качестве своего выходного значения, 
# предполагая, что изображение принадлежит именно этому классу:

model_gender.add(Dense(num_classes))
model_gender.add(Activation('softmax'))

# количество эпох для обучения = ?
# оптимизатор - настроит веса в сети так, чтобы приблизиться к точке с наименьшими потерями. 
# Алгоритм Адама является одним из наиболее часто используемых оптимизаторов


optimizer = 'adam'

# компилируем модель с выбранными параметрами. метрику для оценки = accuracy

model_gender.compile(loss ='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model_gender.summary())

# SEED (симметричный блочный криптоалгоритм на основе Сети Фейстеля)
seed = 21
numpy.random.seed(seed)

# Задаем число эпох обучения
epochs = 100
# epochs = 300

numpy.random.seed(seed)
model_history_gender = model_gender.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=epochs, batch_size=64)

# Точность модели
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# График точности модели
plt.plot(model_history_gender.history['accuracy'])
plt.plot(model_history_gender.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show

# График функции потерь модели
plt.plot(model_history_gender.history['loss'])
plt.plot(model_history_gender.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show

# Сохранение модели
from keras.models import load_model
model_gender.save('model_age.h5')

Работа модели

In [ ]:
# Открываем модель
model_1 = load_model('model_age.h5') # модель распознавания возраста
model_2 = load_model('model_gender.h5') # модель распознавания пола

In [ ]:
# Работа модели на обучающей выборке
def foto_to_array(x_array, size):
    x = np.zeros((len(x_array), size, size))
    i = 0
    for name in x_array:
        image = cv2.imread('result/' + name)
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        x[i] = np.array(gray_img)
        i += 1     
    return x
df = pd.read_csv('foto_frame.csv')
x_new = foto_to_array(df['foto'].values, 128)
y_new = df['age'].values
z_new = df['gender'].values

x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size = 0.25, random_state = 123)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# Случайный выбор фотографии
use_samples = [1, 15, 48, 175]
samples_to_predict = []

# Создание изображения для образца
for sample in use_samples:
    reshaped_image = x_train[sample].reshape((128, 128))
    plt.imshow(reshaped_image)
    plt.show()
    samples_to_predict.append(x_train[sample])

samples_to_predict = np.array(samples_to_predict)

# Генерирование прогноза для выборки
predictions_age = model_1.predict(samples_to_predict)
predictions_gender = model_2.predict(samples_to_predict)

classes_age = np.argmax(predictions_age, axis = 1)
age_name = ['0 - 16', '17-35', '36 - 60', '60+']
age = age_name[classes_age[0]]
print('Возраст: ', age)
classes_gender = np.argmax(predictions_gender, axis = 1)
gender_name = ['Мужчина', 'Женщина']
gender = gender_name[classes_gender[0]]
print('Пол: ', gender)

Работа модели на фотографии нескольких людей

In [ ]:
# Сетевая модель и модель предварительного обучения
faceProto = "weights/opencv_face_detector.pbtxt"
faceModel = "weights/opencv_face_detector_uint8.pb"
ageProto = "weights/deploy_age.prototxt"
ageModel = "weights/age_net.caffemodel"
genderProto = "weights/gender_deploy.prototxt"
genderModel = "weights/gender_net.caffemodel"
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Определить лицо и нарисовать ограничивающую рамку лица
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])  # координаты ограничивающей рамки
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)), 8)
    return frameOpencvDnn, bboxes

# Распознавание пола и возраста
padding = 20
t = time.time()
frame = cv2.imread('имя файла.jpg', cv2.IMREAD_UNCHANGED)             
frameFace, bboxes = getFaceBox(faceNet, frame) 
for bbox in bboxes:
    face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
            max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
    img_resize = cv2.resize(face, (128, 128), interpolation = cv2.INTER_AREA)
    gray_img = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
    
    samples_to_predict = []
    samples_to_predict.append(gray_img)
    samples_to_predict = np.array(samples_to_predict)
    
    predictions_age = model_1.predict(samples_to_predict)
    predictions_gender = model_2.predict(samples_to_predict)

    classes_age = np.argmax(predictions_age, axis = 1)
    age_name = ['0 - 16', '17-35', '36 - 60', '60+']
    age = age_name[classes_age[0]]
    classes_gender = np.argmax(predictions_gender, axis = 1)
    gender_name = ['Male', 'Female']
    gender = gender_name[classes_gender[0]]
         
    label = "{},{}".format(gender, age)
    cv2.putText(frameFace, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.imwrite('новое имя файла.jpg', frameFace)